In [21]:
pip install gurobipy-pandas tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd
import pandas as pd

gppd.set_interactive()

In [6]:
env = gp.Env()
model = gp.Model(env=env)

In [48]:
df = (
    pd.DataFrame({
        "price1": [11, 12, 13, 14, 15],
        "price2": [21, 22, 23, 24, 25],
    })
    .gppd.add_vars(model, name="x")
    .gppd.add_vars(model, name="y")
    .gppd.add_constrs(model, "price1 * x == y", name="c1")
)
print(df.to_markdown())

|    |   price1 |   price2 | x                 | y                 | c1                    |
|---:|---------:|---------:|:------------------|:------------------|:----------------------|
|  0 |       11 |       21 | <gurobi.Var x[0]> | <gurobi.Var y[0]> | <gurobi.Constr c1[0]> |
|  1 |       12 |       22 | <gurobi.Var x[1]> | <gurobi.Var y[1]> | <gurobi.Constr c1[1]> |
|  2 |       13 |       23 | <gurobi.Var x[2]> | <gurobi.Var y[2]> | <gurobi.Constr c1[2]> |
|  3 |       14 |       24 | <gurobi.Var x[3]> | <gurobi.Var y[3]> | <gurobi.Constr c1[3]> |
|  4 |       15 |       25 | <gurobi.Var x[4]> | <gurobi.Var y[4]> | <gurobi.Constr c1[4]> |


In [49]:
print(df["c1"].apply(model.getRow))

0    11.0 x[0] + -1.0 y[0]
1    12.0 x[1] + -1.0 y[1]
2    13.0 x[2] + -1.0 y[2]
3    14.0 x[3] + -1.0 y[3]
4    15.0 x[4] + -1.0 y[4]
Name: c1, dtype: object


In [50]:
for tup in df.itertuples():
    model.chgCoeff(tup.c1, tup.x, tup.price2)

In [51]:
model.update()
df["c1"].apply(model.getRow)

0    21.0 x[0] + -1.0 y[0]
1    22.0 x[1] + -1.0 y[1]
2    23.0 x[2] + -1.0 y[2]
3    24.0 x[3] + -1.0 y[3]
4    25.0 x[4] + -1.0 y[4]
Name: c1, dtype: object

In [36]:
for tup in df.itertuples():
    model.chgCoeff(tup.c1, tup.y, tup.coeff2)

In [37]:
model.update()
df["c1"].apply(model.getRow)

0        21.0 x[0] + y[0]
1    22.0 x[1] + 2.0 y[1]
2    23.0 x[2] + 3.0 y[2]
3    24.0 x[3] + 4.0 y[3]
4    25.0 x[4] + 5.0 y[4]
Name: c1, dtype: object

In [63]:
df2 = pd.concat([df.loc[:2, "x"], df.loc[3:, "y"]]).to_frame(name="v").assign(coeff=0.1, c1=df['c1'])
print(df2)

                   v  coeff                     c1
0  <gurobi.Var x[0]>    0.1  <gurobi.Constr c1[0]>
1  <gurobi.Var x[1]>    0.1  <gurobi.Constr c1[1]>
2  <gurobi.Var x[2]>    0.1  <gurobi.Constr c1[2]>
3  <gurobi.Var y[3]>    0.1  <gurobi.Constr c1[3]>
4  <gurobi.Var y[4]>    0.1  <gurobi.Constr c1[4]>


In [64]:
for tup in df2.itertuples():
    model.chgCoeff(tup.c1, tup.v, tup.coeff)
model.update()

In [65]:
print(df["c1"].apply(model.getRow))

0    0.1 x[0] + -1.0 y[0]
1    0.1 x[1] + -1.0 y[1]
2    0.1 x[2] + -1.0 y[2]
3    24.0 x[3] + 0.1 y[3]
4    25.0 x[4] + 0.1 y[4]
Name: c1, dtype: object
